In [1]:
remotes::install_github("peeter-t2/digar.txts",lib="R_pckg/",upgrade="never")
library(digar.txts,lib.loc="R_pckg/")

* checking for file ‘/tmp/RtmpQw7eu2/remotesb89540c7fb4a/peeter-t2-digar.txts-b87e10a/DESCRIPTION’ ... OK
* preparing ‘digar.txts’:
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
Omitted ‘LazyData’ from DESCRIPTION
* building ‘digar.txts_0.0.0.9100.tar.gz’
Warning in sprintf(gettext(fmt, domain = domain), ...) :
  one argument not used by format 'invalid uid value replaced by that for user 'nobody''



Loading required package: data.table



In [2]:
library(tidyverse)
library(igraph)
install.packages('tidytext', lib='~/R_pckg')
library(tidytext, lib.loc='~/R_pckg')
install.packages('ggraph', lib='~/R_pckg')
library(ggraph, lib.loc='~/R_pckg')

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.8
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::between()   masks data.table::between()
✖ dplyr::filter()    masks stats::filter()
✖ dplyr::first()     masks data.table::first()
✖ dplyr::lag()       masks stats::lag()
✖ dplyr::last()      masks data.table::last()
✖ purrr::transpose() masks data.table::transpose()


Attaching package: ‘igraph’


The following objects are masked from ‘package:dplyr’:

    as_data_frame, groups, union


The following objects are masked from ‘package:purrr’:

    compose, simplify


The following object is masked from ‘package:tidyr’:

    crossing


The following object is masked from ‘package:tibble’:

    as_data_frame


The following objects are masked from ‘package:stats’:

    decompose

In [3]:
all_issues <- get_digar_overview()

[1] "Issue metadata read"


In [4]:
stopsonad <- read.table("estonian-stopwords-lemmas.txt", header = TRUE, sep = "\t", encoding = "UTF-8", stringsAsFactors = T)

In [5]:
rootsi_ajalehed1 <- all_issues %>%
    filter(DocumentType=="NEWSPAPER") %>%
    filter(year>1943&year<1961) %>%
    filter(keyid=="estdagbladet" | keyid=="eestiteatajastock" | keyid=="stockholmstid" | keyid=="teatajapoliit" | keyid=="valiseesti")

In [6]:
searchterm <- "balti"
searchfile <- "rootsi_trigrammid1.txt"
do_subset_search(searchterm=searchterm, searchfile=searchfile, rootsi_ajalehed1, searchtype="lemmas", source="pages")

In [7]:
concs1 <- fread("rootsi_trigrammid1.txt",header=F,sep="\t", quote="")[,.(id=V1,lemmas=V2)]
concs1[,DocumentID:=str_extract(id,"[^\\.]+")]
fileids_w_meta_concs1 <- concs1 %>% select(id, DocumentID) %>% left_join(rootsi_ajalehed1 %>% select(keyid, year, month, DocumentID, zippath_sections, zippath_pages), by=c("DocumentID"))

In [8]:
rootsi_concs1 <- get_concordances(searchterm=searchterm,texts=concs1,before=50,after=50,txt="lemmas",id="id")

In [9]:
rootsi_concs_4460 <- rootsi_concs1[, !c("id","nr","begin","end", "DocumentID")]

write.table(rootsi_concs_4460, "4460.txt", sep="\t", quote = F, row.names = F)

In [10]:
rootsi_concs_4460_2 <- str_replace_all(rootsi_concs_4460$context, "\\|", "_")

rootsi_concs_4460_3 <- as.data.frame(rootsi_concs_4460_2, header = TRUE)
colnames(rootsi_concs_4460_3) <- "context"
head(rootsi_concs_4460_3, 10)

,context
,<chr>
1,"uma_kutsutud eesti , läti ja leedu rahvas esindus balti komitee . eestlane esindaja võtma komitee juhtimi"
2,d Moskva ka Türgi välispoliitika juht Saraeoglu . balti maa esindaja olema sundima_sunnitud andma_antud o
3,"elgia Taani Kopenhaagen , 28. juuli . Kopenhaagen balti põgenik laager jõudma eestlane , kes oma põgenemi"
4,kes kõik olema nn . Baltikum laager . üldse olema balti riik kodanik seal umbes 100 inimene . teine eestl
5,"valdama_avaldatud . Unrra poolt teatama veel , et balti riik kodaku_kodakud , kes oma kodumaa tagasi ei t"
6,nike praktiseerimine võimalus laiendama . Arste - balti põgenik tahtma rakendama Rootsi Punane_Punas Rist
7,"rd kuulma , et Potsdam võima päevakord kerkima ka balti riik tulevik küsimus . teatavasti ei olema Inglis"
8,e kodanik . 970 see olema haige ja ülejäänu olema balti maa . Inglis ametivõim ei vaatama aga tema kui ’‘
9,aheline organisatsioon esindaja Poolamaa ja teine balti maa ; 4 ) nõukogu vene sõjavägi lahkumine Poolaj


In [11]:
trigrams_separated1 <- rootsi_concs_4460_3 %>%
  unnest_tokens(trigram, context, token = "ngrams", n = 3, n_min = 3) %>%
  separate(trigram, c("word1", "word2", "word3"), sep = " ")

In [12]:
trigrams_filtered1 <- trigrams_separated1 %>%
  filter(!word1 %in% stopsonad$word) %>%
  filter(!word2 %in% stopsonad$word) %>%
  filter(!word3 %in% stopsonad$word)

In [13]:
trigram_counts1 <- trigrams_filtered1 %>% 
  count(word1, word2, word3, sort = TRUE)

In [14]:
head(trigram_counts1, 30)

,word1,word2,word3,n
,<chr>,<chr>,<chr>,<int>
1,kolm,balti,riik,294
2,balti,humanist,ühing,179
3,balti,riik,küsimus,160
4,kolm,balti,rahvas,160
5,balti,riik,iseseisvus,126
6,balti,riik,esindaja,121
7,balti,riik,annekteerimine,113
8,balti,riik,saatus,100
9,balti,riik,kodanik,92


In [15]:
rootsi_ajalehed2 <- all_issues %>%
    filter(DocumentType=="NEWSPAPER") %>%
    filter(year>1960&year<1985) %>%
    filter(keyid=="estdagbladet" | keyid=="eestiteatajastock" | keyid=="stockholmstid" | keyid=="teatajapoliit" | keyid=="valiseesti")

In [16]:
searchfile <- "rootsi_trigrammid2.txt"
do_subset_search(searchterm=searchterm, searchfile=searchfile, rootsi_ajalehed2, searchtype="lemmas", source="pages")

In [17]:
concs2 <- fread("rootsi_trigrammid2.txt",header=F,sep="\t", quote="")[,.(id=V1,lemmas=V2)]
concs2[,DocumentID:=str_extract(id,"[^\\.]+")]
fileids_w_meta_concs2 <- concs2 %>% select(id, DocumentID) %>% left_join(rootsi_ajalehed2 %>% select(keyid, year, month, DocumentID, zippath_sections, zippath_pages), by=c("DocumentID"))

In [18]:
rootsi_concs2 <- get_concordances(searchterm=searchterm,texts=concs2,before=50,after=50,txt="lemmas",id="id")

In [19]:
rootsi_concs_6184 <- rootsi_concs2[, !c("id","nr","begin","end", "DocumentID")]

write.table(rootsi_concs_6184, "6184.txt", sep="\t", quote = F, row.names = F)

In [20]:
rootsi_concs_6184_2 <- str_replace_all(rootsi_concs_6184$context, "\\|", "_")

rootsi_concs_6184_3 <- as.data.frame(rootsi_concs_6184_2, header = TRUE)
colnames(rootsi_concs_6184_3) <- "context"
head(rootsi_concs_6184_3, 10)

,context
,<chr>
1,"itte kuidagi kaasuma » ida kest ja kah » t & gt ■ balti riik viigivnlds » annektmoonl ja S , KHtu Inkorpo"
2,"foorum , sama ! aeg andma oma ahi Eesti ja teine balti rahvas võõrandama õigus laastamine . mina vlme sa"
3,"õigus laastamine . mina vlme samuti rõhutama , et balti riik iseseisvus olema oluline balti meri vabadus"
4,"rõhutama , et balti riik iseseisvus olema oluline balti meri vabadus ja see kogu Euroopa julgeolek vaba n"
5,"ku * kontsert tolmuma Chicago *, IndianspolisNi ( balti kontserttihlngu kontsert sari *}, Bo * io # is ba"
6,"ti kontserttihlngu kontsert sari *}, Bo * io # is balti kontsert sari * ( 20 ,’ jaanuar }, Bartfort *. Ne"
7,"de , mis nähtus , et mööduma_möödunud aasta olema balti komitee juba 38 aasta kestma_kestnud tegeras olem"
8,"as olema_olnud kõige aktiivsem ja tulemusrikkam , balti põgenik Eesti jöuhtpuu i Vake ^ ttaja ees . 1 . w"
9,olema min . V. Sidzikauskaa . see organ andma ka balti komitee kaudud_kauduma väiksem majandusiik_majand


In [21]:
trigrams_separated2 <- rootsi_concs_6184_3 %>%
  unnest_tokens(trigram, context, token = "ngrams", n = 3, n_min = 3) %>%
  separate(trigram, c("word1", "word2", "word3"), sep = " ")

In [22]:
trigrams_filtered2 <- trigrams_separated2 %>%
  filter(!word1 %in% stopsonad$word) %>%
  filter(!word2 %in% stopsonad$word) %>%
  filter(!word3 %in% stopsonad$word)

In [23]:
trigram_counts2 <- trigrams_filtered2 %>% 
  count(word1, word2, word3, sort = TRUE)

In [24]:
head(trigram_counts2, 30)

,word1,word2,word3,n
,<chr>,<chr>,<chr>,<int>
1,balti,riik,küsimus,362
2,de,baltiska_baltiskas,staterna,316
3,kolm,balti,riik,258
4,balti,riik,annekteerimine,191
5,balti,riik,okupeerimine,162
6,balti,riik,iseseisvus,145
7,balti,komitee,esimees,132
8,tunnustama,balti,riik,121
9,balti,riik,saatus,119


In [25]:
rootsi_ajalehed3 <- all_issues %>%
    filter(DocumentType=="NEWSPAPER") %>%
    filter(year>1984&year<1996) %>%
    filter(keyid=="estdagbladet" | keyid=="eestiteatajastock" | keyid=="stockholmstid" | keyid=="teatajapoliit" | keyid=="valiseesti")

In [26]:
searchfile <- "rootsi_trigrammid3.txt"
do_subset_search(searchterm=searchterm, searchfile=searchfile, rootsi_ajalehed3, searchtype="lemmas", source="pages")

In [27]:
concs3 <- fread("rootsi_trigrammid3.txt",header=F,sep="\t", quote="")[,.(id=V1,lemmas=V2)]
concs3[,DocumentID:=str_extract(id,"[^\\.]+")]
fileids_w_meta_concs3 <- concs3 %>% select(id, DocumentID) %>% left_join(rootsi_ajalehed3 %>% select(keyid, year, month, DocumentID, zippath_sections, zippath_pages), by=c("DocumentID"))

In [28]:
rootsi_concs3 <- get_concordances(searchterm=searchterm,texts=concs3,before=50,after=50,txt="lemmas",id="id")

In [29]:
rootsi_concs_8595 <- rootsi_concs3[, !c("id","nr","begin","end", "DocumentID")]

write.table(rootsi_concs_8595, "8595.txt", sep="\t", quote = F, row.names = F)

In [30]:
rootsi_concs_8595_2 <- str_replace_all(rootsi_concs_8595$context, "\\|", "_")

rootsi_concs_8595_3 <- as.data.frame(rootsi_concs_8595_2, header = TRUE)
colnames(rootsi_concs_8595_3) <- "context"
head(rootsi_concs_8595_3, 10)

,context
,<chr>
1,"aruanne konstateerima , et vene keel oskus olema baltikumi » üha enam USA-rapori nimetama otseselt Öigus"
2,", N.liit . ( VES ). Ilo Käbin - mees Armeemmiseum balti näitus taga kindlasti olema palju kaasmaalane kas"
3,"sti tutvustama Baltikum rahvas , Baltimaa kui Ita balti riik ajalugu , eriti kuna see olema Rootsi võrdle"
4,"_koht a . 1977. aasta hiljem võitma sama meeskond balti karikas , mis olema vast kõige suurem edu . hilje"
5,"amisc ” näol . Berliin olema Idaministeeriu - mis balti küsimus osakond juhataja dr , Peter von Kleist po"
6,"esti rahvusnõukogu esimees Mihkel Mathfesen EKN , balti komitee ja Eesti komitee traditsioniliscl uueaast"
7,ueaasta päev kogunema vastuvõtt sadakond eesti ja balti organisatsioon esindaja ning rootsipoolne külalin
8,"lismaa koondis , Mihkel Mathiesen-Rahvusnõukogu , balti komitee , balti arhiiv ja ajaleht ” Teataja ” aua"
9,"Mihkel Mathiesen-Rahvusnõukogu , balti komitee , balti arhiiv ja ajaleht ” Teataja ” auaadress , Valdo N"


In [35]:
trigrams_separated3 <- rootsi_concs_8595_3 %>%
  unnest_tokens(trigram, context, token = "ngrams", n = 3, n_min = 3) %>%
  separate(trigram, c("word1", "word2", "word3"), sep = " ")

In [36]:
trigrams_filtered3 <- trigrams_separated3 %>%
  filter(!word1 %in% stopsonad$word) %>%
  filter(!word2 %in% stopsonad$word) %>%
  filter(!word3 %in% stopsonad$word)

In [37]:
trigram_counts3 <- trigrams_filtered3 %>% 
  count(word1, word2, word3, sort = TRUE)

In [39]:
head(trigram_counts3, 30)

,word1,word2,word3,n
,<chr>,<chr>,<chr>,<int>
1,kolm,balti,riik,233
2,balti,riik,iseseisvus,132
3,de,baltiska_baltiskas,staterna,118
4,balti,riik,küsimus,77
5,balti,riik,annekteerimine,68
6,balti,riik,okupeerimine,66
7,balti,vabadusliit,esimees,62
8,vabadusliit,esimees,avo,56
9,kolm,balti,rahvas,52
